In [ ]:
import os
from utils import *
import optuna

In [ ]:
data, usermap, itemmap, users = load_data2()
data_train, data_test, data_val = split_data2(data, 0, 0.2)

In [ ]:
study_name = "ItemKNNCF"
study = optuna.create_study(
    study_name=study_name,
    storage=get_database_url(),
    load_if_exists=True,
    direction="maximize",
)

In [ ]:
from Recommenders.KNN.UserKNNCFRecommender import ItemKNNCFRecommender

def objective(trial):
    topK = trial.suggest_int('topK', 0, 1000)
    shrink = trial.suggest_int('topK', 0, 1000)
    similarity = trial.suggest_categorical('similarity', ['cosine', 'jaccard', 'asymmetric', 'dice', 'tversky'])
    normalize = trial.suggest_categorical('normalize', [True, False])
    feature_weighting = trial.suggest_categorical('feature_weighting', ['BM25', 'TF-IDF', 'none'])
    
    recommender = ItemKNNCFRecommender(data_train, verbose=False)
    recommender.fit(
        topK=topK,
        shrink=shrink,
        similarity=similarity,
        normalize=normalize,
        feature_weighting=feature_weighting,
    )
    _, _, ev_map, _, _ = evaluator(recommender, data_train, data_val)
    
    return ev_map

study.optimize(objective, n_trials=300)

In [ ]:
study.best_params

In [ ]:
from Recommenders.KNN.ItemKNNCFRecommender import ItemKNNCFRecommender

recommender = ItemKNNCFRecommender(data_train, verbose=False)
recommender.fit(**study.best_params)

In [ ]:
pr, re, ev_map, ev_ndcg, ev_auc = evaluator(recommender, data_train, data_val)
print("Precision: {:.8f}".format(pr))
print("Recall: {:.8f}".format(re))
print("MAP: {:.8f}".format(ev_map))

In [ ]:
submission2(recommender, users, usermap, itemmap, data_train)